In [1]:
# Tensorflow and Keras should be at their latest version for good accuracy.
# Versions used here are : 
#               tensorflow Version - 2.2.0
#               keras Version - 2.4.3
# If the system is GPU enabled, CUDA and CUDnn should be installed and added to path.

In [2]:
# All necessary imports
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import Adam, RMSprop
import glob
import numpy as np

In [3]:
# Check versions
import keras
import tensorflow 
print(keras.__version__)
print(tensorflow.__version__)

2.4.3
2.2.0


In [4]:
#Here Image dimensions are in the ratio 3:2. 
#It can be changed based on object shapes.

img_width, img_height = 450, 300

In [5]:
train_data_dir = 'images/train' #train images path
test_data_dir = 'images/test' #test images path
epochs = 20
batch_size = 2  

In [6]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [7]:
# Training Network (Model)

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [8]:
# Hyperparameters
sgd = RMSprop(lr=0.0005)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [9]:
# Print Summary of Model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 448, 298, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 448, 298, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 224, 149, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 222, 147, 64)      18496     
_________________________________________________________________
activation_1 (Activation)    (None, 222, 147, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 73, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 71, 32)       1

In [10]:
# Using ImageDataGenerator to use "flow_from_directory" function which automatically..
#..labels the binary classification images.
# All identical class images should be kept in seperate folders under one train folder.
# Thus for binary classification train folder will have two sub folders.
# No need to rescale the pixel values of images

train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 48 images belonging to 2 classes.


In [11]:
#Training the model
model.fit(
    train_generator,
    epochs=epochs)

Epoch 1/20
24/24 [==============================] - 10s 428ms/step - loss: 42.6767 - accuracy: 0.7292
Epoch 2/20
24/24 [==============================] - 10s 417ms/step - loss: 0.2705 - accuracy: 0.9792
Epoch 3/20
24/24 [==============================] - 10s 429ms/step - loss: 0.0195 - accuracy: 0.9792
Epoch 4/20
24/24 [==============================] - 10s 417ms/step - loss: 0.0074 - accuracy: 1.0000
Epoch 5/20
24/24 [==============================] - 10s 423ms/step - loss: 9.6312 - accuracy: 0.8750
Epoch 6/20
24/24 [==============================] - 10s 419ms/step - loss: 6.0652e-05 - accuracy: 1.0000
Epoch 7/20
24/24 [==============================] - 10s 416ms/step - loss: 4.0953 - accuracy: 0.9375
Epoch 8/20
24/24 [==============================] - 10s 415ms/step - loss: 2.7740e-09 - accuracy: 1.0000
Epoch 9/20
24/24 [==============================] - 10s 425ms/step - loss: 0.7918 - accuracy: 0.9792
Epoch 10/20
24/24 [==============================] - 10s 416ms/step - loss: 6.3415

In [12]:
#Saving weights
model.save_weights('final_orientation1.h5')

In [13]:
# Predict the output for all test images
test_names = sorted(glob.glob("images/test/test_folder/*.png"))
for name in test_names :
  img = load_img(name,target_size=(450,300))
  input_arr = img_to_array(img)
  input_arr = np.array([input_arr])  # Convert single image to a batch.
  print(model.predict(input_arr))

[[1.]]
[[0.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[0.]]
[[1.]]
[[0.]]
[[1.]]


In [14]:
# ImageDataGenerator function should not be used for predicting output for test images as it may give...
# ... wrong results. Reason for this can be found out.